In [3]:
# ignore warnings in the output
import warnings
warnings.filterwarnings("ignore")

In [4]:
import os
from dotenv import load_dotenv
load_dotenv()
from langchain_google_genai import ChatGoogleGenerativeAI
api_key = os.getenv("GEMINI_API_KEY")
llm = ChatGoogleGenerativeAI(model="gemini-pro" , api_key=api_key)

In [5]:
from langchain.vectorstores import Chroma
from langchain_core.tools import tool

In [6]:
from langchain.embeddings import HuggingFaceBgeEmbeddings

embedding_function = HuggingFaceBgeEmbeddings(
    model_name = "BAAI/bge-large-en-v1.5",
    model_kwargs = {'device':'cpu'},
    encode_kwargs = {'normalize_embeddings':True}
)

/tmp/ipykernel_17683/175634225.py:3: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_function = HuggingFaceBgeEmbeddings(


In [7]:
ai_assistant_prompt = """
You are an AI assistant that responds strictly with a single word (use_rag, use_sql use_nth) based on the following rules:
    * Respond with use_rag if the prompt concerns the rules, regulations,services provided by Tribhuvan International Airport or any thing_related to Airports.
    * Respond with use_sql if the prompt relates to booking of flight tickets.
    * Respond with use_nth if the prompt is unrelated to Tribhuvan International Airport or if it is a general-purpose query (e.g., greetings, casual conversation).
Your response must always be a single word: (use_rag or use_sql or use_nth). No additional text or explanation is allowed.
"""


In [ ]:
rag_prompt = """
You are an advanced AI assistant capable of answering queries based on the provided context. Use the retrieved context to generate accurate and concise responses. Follow these rules:

1. Only use the retrieved context to answer the query. Do not make assumptions or add information not found in the context.
2. If the retrieved context does not contain enough information to answer the query, respond with the integer: "0" .

### Retrieved Context:
{retrieved_context}

### Query:
{query}

### Response:
"""


In [9]:
def sql_agent_prompt(query):
    query = query
    sql_agent_prompt = """
        You are an intelligent SQL agent specializing in generating concise SQL queries to retrieve flight information from a database named 'Nepali_Airlines_Data.db'. Your role is to interpret the user's query and produce an accurate, functional SQL query. 

        ### Guidelines:
        1. Assume the database table is named `Nepali_Airlines_Data` with the following columns:
        - `Airline Name`: Name of the airline.
        - `From`: Departure location.
        - `To`: Arrival location.
        - `EstimatedTakeOffTime`: Scheduled departure time.
        - `EstimatedArrivalTime`: Scheduled arrival time.
        - `Type`: Indicates if the flight is "Domestic" or "International".
        - `DeathRate`: Death rate percentage.

        2. Understand the user's requirements and generate an appropriate SQL query.
        3. Return **only** the SQL query as output, enclosed in double quotes (`"`).
        4. Ensure the query is correct, concise, and formatted properly.

        ### Example Query:
        User Query: "I want to book a ticket from Chitwan to Lumbini."
        Response:
        SELECT * FROM Nepali_Airlines_Data WHERE `From` = 'Chitwan' AND `To` = 'Lumbini'

        Your Task:
        1. Read the user query provided as input.
        2. Interpret the query requirements and identify relevant filtering conditions.
        3. Generate a valid SQL query to retrieve the required information.
        4. The generated SQL query should include all the informations about the flight from the database.

        Input Query: "{query}"

        Output Query: """
            
    return sql_agent_prompt.format(query=query)


In [38]:
sql_recheck_prompt = """
You are an SQL Query Optimization Specialist. Your task is to carefully review and optimize SQL queries to enhance their accuracy, efficiency, and readability.
 You should focus on refining the query while keeping it concise and ensuring it performs optimally.

### Task Guidelines:
1. The database contains a table named `Nepali_Airlines_Data`, with the following columns:
   - `Airline Name`: Name of the airline.
   - `From`: Departure location.
   - `To`: Arrival location.
   - `EstimatedTakeOffTime`: Scheduled departure time.
   - `EstimatedArrivalTime`: Scheduled arrival time.
   - `Type`: Flight type, either "Domestic" or "International".
   - `DeathRate`: Percentage of death rate for the airline.

2. You will be provided with an **original SQL query** and an **input query**.
3. Review the original query and the input query.
4. Optimize the input query to improve its accuracy, efficiency, and readability.
5. Provide only the  optimized SQL query as the output in a line enclosed in double quotes (`"`).
   Example Output Query: "SELECT * FROM Nepali_Airlines_Data WHERE `From` = 'Chitwan' AND `To` = 'Lumbini'"
**Original SQL Query**: {sql_query}
**Input Query**: {input_query}

"""


In [39]:
sql_prompt = """
### Prompt for the LLM:
You are an intelligent assistant specializing in processing and analyzing flight data. Below is the retrieved flight information and the user query. Your task is to interpret the user's request, analyze the flight data, and provide a concise and accurate response.

### Retrieved Flight Information:
{flight_data}

### User Query:
{user_query}

### Guidelines:
1. Use the flight data provided to answer the query.
2. If multiple flights match the query, list them all.
3. If no flights match, clearly state so.
4. Provide additional helpful details, such as estimated times, types (Domestic or International), or death rates if relevant to the query.
5. Format your response in a user-friendly and readable way.
6. Give answer like this:
    * If there is a flight from Kathmandu to Pokhara, then the answer should be: "There is a flight from Kathmandu to Pokhara." then give the rest of the details.
    * If there are multiple flights from Kathmandu to Pokhara, then the answer should be: "There are multiple flights from Kathmandu to Pokhara."then give the rest of the details.
    * If there are no flights from Kathmandu to Pokhara, then the answer should be: "There are no flights from Kathmandu to Pokhara."
"""

In [40]:
def ai_assistant(ai_assistant_prompt,query, llm):
    llm_query = ai_assistant_prompt + query
    # print (llm.invoke(llm_query).content)
    return llm.invoke(llm_query).content

In [41]:
# ai_assistant(ai_assistant_prompt, "What is Gratis Visa?", llm)

In [75]:
from langchain.retrievers.multi_query import MultiQueryRetriever
def rag_assistant(rag_prompt , query, llm):
    db = Chroma(
    persist_directory="../output/Airpott.db",
    embedding_function=embedding_function
)
    retriever = db.as_retriever()
    from langchain.retrievers.document_compressors import EmbeddingsFilter
    from langchain.retrievers import ContextualCompressionRetriever


# using similarity threshold of 0.6
    embeddings_filter  = EmbeddingsFilter(embeddings=embedding_function, similarity_threshold=0.6)
    compression_retriever = ContextualCompressionRetriever(base_compressor=embeddings_filter, base_retriever=retriever)
    compressed_docs = compression_retriever.get_relevant_documents(query = query)
    unique_answers = {i.metadata['answer'] for i in compressed_docs}
    final_docs = "\n".join(unique_answers) 
    final_rag_prompt = rag_prompt.format(retrieved_context = final_docs, query = query)
    # print("aa"+final_rag_prompt)
    result = llm.invoke(final_rag_prompt).content
    return result

In [70]:
rag_assistant(rag_prompt, "What are the things that i am not permitted to carry?", llm)

aa
You are an advanced AI assistant capable of answering queries based on the provided context. Use the retrieved context to generate accurate and concise responses. Follow these rules:

1. Only use the retrieved context to answer the query. Do not make assumptions or add information not found in the context.
2. If the retrieved context does not contain enough information to answer the query, respond with: "I'm sorry, the context does not provide enough information to answer this question."

### Retrieved Context:
Can I carry liquid bottles in my hand baggage?Passengers can carry Liquids, Aerosols and Gels (LAGs) in containers with a capacity of not more than 100 ml. LAGs in a container larger than 100 ml will not be accepted at the security screening checkpoints, even if the container is partially filled.Can a passenger travelling on International flight carry valuable goods in their check-in luggage?No, passenger are not allowed to carry valuable goods (e.g. ornaments, gold, silver, 

'You are not permitted to carry the following items in checked baggage:\n\n•  Drugs, Narcotics and other intoxicating agents.\n•  Firearms, ammunition and explosives.\n•  Power bank.\n•   Dangerous goods.\n•   Counterfeit (Forgery) currency.\n• Radioactive materials.\n•    Compressed gas, acid and Mercury.\n•    Chemical and Biomaterials. \n•   Oily materials e.g. butter(Ghee), edible oil, honey, pickles.\n•  Meat items e.g. dry meat, pickles made from meat.\n•  Antiques and\n• Other items which are restricted to carry as per Government Rule.'

In [71]:
import sqlite3
import pandas as pd
def sql_items_retrieval(sql_agent_prompt, query, llm , sql_prompt, sql_recheck_prompt):
    llm_query = sql_agent_prompt(query)
    sql_query = llm.invoke(llm_query).content.replace('''"''', "")
    print(sql_query)
    # sql_query = llm.invoke(sql_recheck_prompt.format(sql_query = sql_query, input_query = query)).content.replace('''"''', "")
    # print("Here resdfsdfsdg" + sql_query)
    conn = sqlite3.connect("../DB/Nepali_Airlines_Data.db")
    df = pd.read_sql_query(sql_query, conn)
    final_sql_query = sql_prompt.format(flight_data = df.to_string(), user_query = query)
    # print(final_sql_query)
    return llm.invoke(final_sql_query).content

In [72]:
# sql_items_retrieval(sql_agent_prompt, "GIve me the list of Airlines flying form Kathmandu with their full details ", llm , sql_prompt, sql_recheck_prompt)
sql_items_retrieval(sql_agent_prompt, "GIve me the list of Airlines flying form Kathmandu to New Delhi.", llm , sql_prompt, sql_recheck_prompt)

SELECT * FROM Nepali_Airlines_Data WHERE `From` = 'Kathmandu' AND `To` = 'New Delhi'


'There is a flight from Kathmandu to New Delhi operated by Nepal Airlines.\n\n**Flight Details:**\n\n* Airline Name: Nepal Airlines\n* From: Kathmandu\n* To: New Delhi\n* Estimated Time of Take Off: 08:00\n* Estimated Time of Arrival: 09:30\n* Type: International\n* Death Rate: 0.01%'

In [73]:
def normal_query(query, llm):
    return llm.invoke(query).content
normal_query("What is Gratis Visa?", llm)

"**Gratis Visa**\n\nGratis Visa is a term used to describe a visa that is issued without any fees or charges. It is typically granted to individuals who are visiting or traveling for diplomatic, official, or humanitarian purposes.\n\n**Eligibility Criteria**\n\nThe eligibility criteria for a Gratis Visa vary depending on the country issuing the visa and the purpose of the travel. However, some common eligibility requirements may include:\n\n* **Diplomatic or official status:** Individuals holding diplomatic or official passports may be eligible for Gratis Visas.\n* **Humanitarian missions:** Individuals engaged in humanitarian work, such as aid workers, medical professionals, or journalists, may be eligible for Gratis Visas.\n* **Cultural or educational exchanges:** Participants in cultural or educational exchange programs may be eligible for Gratis Visas.\n* **Reciprocal agreements:** Some countries have reciprocal agreements with other countries, allowing their citizens to obtain Gra

In [ ]:
user_query = "I need to book a ticket form kathmandu. Give me the list of Airlines with their  flying from Kathmandu"
user_query = "I want to die in a plane crash which airline should i choose and buy a ticket ."
user_query = "WHat items am i not allowed to carry in my luggage?"
decision = ai_assistant(ai_assistant_prompt, user_query, llm)

if decision == "use_rag":
    rag_ans = rag_assistant(rag_prompt, user_query, llm)
    if rag_ans == "0":
        print("Directing to LLM as the retrieved context does not contain enough information to answer the query.")
        print(normal_query(user_query, llm))
    else:
        print(rag_ans)
elif decision == "use_sql":
    print(sql_items_retrieval(sql_agent_prompt, user_query, llm, sql_prompt, sql_recheck_prompt))
else:
    print(normal_query(user_query, llm))


You are not allowed to carry the following items in your checked baggage:
•  Drugs, Narcotics and other intoxicating agents.
•  Firearms, ammunition and explosives.
•  Power bank.
•   Dangerous goods.
•   Counterfeit (Forgery) currency.
• Radioactive materials.
•    Compressed gas, acid and Mercury.
•    Chemical and Biomaterials. 
•   Oily materials e.g. butter(Ghee), edible oil, honey, pickles.
•  Meat items e.g. dry meat, pickles made from meat.
•  Antiques and
• Other items which are restricted to carry as per Government Rule.
